In [1]:
from transformers import RobertaForSequenceClassification
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RobertaForSequenceClassification.from_pretrained('roberta-base')
model.load_state_dict(torch.load('best.pth'))
model.to(device)

c:\Users\wzy\anaconda3\envs\nlu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [2]:
import pandas as pd

eval_data = pd.read_csv('Data/dev.csv', encoding='utf-8')
eval_data.head()

,premise,hypothesis,label
0,Mon Dieu!,This person is speaking English.,0
1,"He really shook up my whole mindset, Broker sa...","His mindset never changed, Broker said.",0
2,Patients were asked to place themselves on a r...,Most patients rated themselves as a 5 on the s...,1
3,I managed to pick-pocket someone next to the s...,I stole someone's wallet near the concession s...,1
4,Forty comments were received and considered pr...,The decisions regarding the issuance of the fi...,1


In [3]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [14]:
premise = eval_data['premise'][3126]
hypothesis = eval_data['hypothesis'][3126]
label = int(eval_data['label'][3126])
print(premise)
print(hypothesis)
encoded_premise = tokenizer(premise,
                            "have shoes", 
                            padding='max_length', 
                            truncation=True, 
                            max_length=256, 
                            return_tensors='pt')


input_ids = encoded_premise['input_ids'].to(device)
attention_mask = encoded_premise['attention_mask'].to(device)

outputs = model(input_ids, attention_mask=attention_mask)
preds = torch.argmax(outputs.logits, dim=1)
print(preds)

Tony  Shoes (so Clinton will have Shoes and Socks).
nan
tensor([1], device='cuda:0')


In [4]:
prediction = []
true_labels = []
for i in range(len(eval_data)):
    premise = eval_data['premise'][i]
    hypothesis = eval_data['hypothesis'][i]
    label = int(eval_data['label'][i])
    try:
        encoded_premise = tokenizer(premise,
                                    hypothesis, 
                                    padding='max_length', 
                                    truncation=True, 
                                    max_length=256, 
                                    return_tensors='pt')
    except:
        print("Error at index:", i)
        print("Premise:", premise)
        print("Hypothesis:", hypothesis)
        print("Label:", label)
        if hypothesis == "nan":
            prediction.append(1)
            true_labels.append(label)
        continue

    input_ids = encoded_premise['input_ids'].to(device)
    attention_mask = encoded_premise['attention_mask'].to(device)

    outputs = model(input_ids, attention_mask=attention_mask)
    preds = torch.argmax(outputs.logits, dim=1)
    prediction.append(preds.item())
    true_labels.append(label)


Error at index: 3126
Premise: Tony  Shoes (so Clinton will have Shoes and Socks).
Hypothesis: nan
Label: 1
Error at index: 3970
Premise: Saint-Germain-des-Pr??s
Hypothesis: nan
Label: 1


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


In [5]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

val_acc = accuracy_score(true_labels, prediction)
f1 = f1_score(true_labels, prediction, average='macro')
precision = precision_score(true_labels, prediction, average='macro')
recall = recall_score(true_labels, prediction, average='macro')

print(f'Acc: {val_acc}, F1: {f1}, Precision: {precision}, Recall: {recall}')

Acc: 0.8807720861172977, F1: 0.8806179535006453, Precision: 0.88073179727605, Recall: 0.8805284630929098


In [6]:
correct = 0
for i in range(len(prediction)):
    if prediction[i] == true_labels[i]:
        correct += 1

print(correct / len(prediction))

0.8807720861172977
